<a href="https://colab.research.google.com/github/mochi1214/openCV_test/blob/main/opencv_playmp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2
import numpy as np
import subprocess
import os
import time
import base64
import IPython.display as dp

# 取得影像來源：
cap = cv2.VideoCapture('h3.mp4')

# 獲取影片寬度和高度
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 影片寫入物件
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (width, height))

if not out.isOpened():
    raise Exception("Could not open output video file")

while True:
    ret, frame = cap.read()
    if ret:

        # 將圖像轉換到HSV色彩空间
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # 創建藍色的mask（遮罩）
        mask = cv2.inRange(hsv, (110,50,50), (130,255,255))

        # 先膨脹後侵蝕讓圖片乾淨
        mask = cv2.erode(mask, np.ones((2, 2)))
        mask = cv2.dilate(mask, np.ones((42, 42)))
        mask = cv2.erode(mask, np.ones((38, 38)))

        # 尋找輪廓
        contours, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # 繪製輪廓
        cv2.drawContours(frame, contours, -1, (0,0,255), 3)

        # 寫入畫面
        out.write(frame)

    else:
        break

# 釋放資源並關閉窗口
cap.release()
out.release()

while not os.path.exists("output.mp4"):
    print("Waiting for output.mp4 to be created...")
    time.sleep(1)

print("output.mp4 exists, proceeding with ffmpeg...")
subprocess.call(['ffmpeg', '-i', 'output.mp4', '-vcodec', 'libx264', 'output_h264.mp4'])

# 讀取轉檔後的影片
with open("output_h264.mp4", "rb") as f:
  d = base64.b64encode(f.read()).decode()

# 顯示影片
dp.HTML(
    f'<video src = "data:video/mp4;base64, {d}" controls>'
)


output.mp4 exists, proceeding with ffmpeg...
